### Triplet Loss

As first introduced in the FaceNet paper, TripletLoss is a loss function that trains a neural network to closely embed features of the same class while maximizing the distance between embeddings of different classes. To do this an anchor is chosen along with one negative and one positive sample.

The loss function is described as a Euclidean distance function:

![function](https://user-images.githubusercontent.com/18154355/61484709-7589b800-a96d-11e9-9c3c-e880514af4b7.png)

Where A is our anchor input,  P is the positive sample input,  N is the negative sample input, and alpha is some margin you use to specify when a triplet has become too "easy" and you no longer want to adjust the weights from it.

### SemiHard Online Learning

As shown in the paper, the best results are from triplets known as "Semi-Hard". These are defined as triplets where the negative is farther from the anchor than the positive, but still produces a positive loss. To efficiently find these triplets you utilize online learning and only train from the Semi-Hard examples in each batch. 

In [1]:
pip install -q -U tensorflow-addons

     |████████████████████████████████| 1.1MB 8.8MB/s 


In [2]:
import io

import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

In [3]:
# Data prep

def _normalize_img(img, label):
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)

train_dataset, test_dataset = tfds.load(name = "mnist", split = ['train', 'test'], as_supervised = True)

# Build your input pipelines
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.map(_normalize_img)

test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.map(_normalize_img)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [4]:
# Prep Model

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters = 64, kernel_size = 2, padding = 'same', activation = 'relu', input_shape = (28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size = 2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters = 32, kernel_size = 2, padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size = 2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis = 1)) # L2 normalize embeddings

])

In [5]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(0.001),
    loss = tfa.losses.TripletSemiHardLoss())

In [6]:
model.fit(
    train_dataset,
    epochs = 5)

Epoch 1/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.5951
Epoch 2/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.4513
Epoch 3/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.4191
Epoch 4/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.4018
Epoch 5/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3927


In [7]:
# Evaluate and save

results = model.predict(test_dataset)

np.savetxt("vecs.tsv", results, delimiter = '\t')

out_m = io.open('meta.tsv', 'w', encoding = 'utf-8')
for img, labels in tfds.as_numpy(test_dataset):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()